In [1]:
import datetime
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Activation, Dropout, Flatten, Dense, Conv2D, MaxPool2D

import tensorflow as tf
print("TF version: ", tf.__version__)

In [2]:
device_name = tf.test.gpu_device_name()
if "GPU" not in device_name:
    print("No")
else:
    print(device_name)

In [ ]:
print(tf.test.is_gpu_available())

In [2]:
train_df = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Train.csv')
train_df.describe()

In [3]:
train_df = train_df.drop(['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2'], axis = 1)
train_df.head()

In [4]:
train_df['ClassId'].value_counts().plot.bar(figsize=(20, 10))
train_df['ClassId'].value_counts().median()

In [5]:
filenames = ['/kaggle/input/gtsrb-german-traffic-sign/' + fname for fname in train_df['Path']]
filenames[:10]

In [6]:
labels = train_df['ClassId'].to_numpy()
labels

In [7]:
unique_signs = np.unique(labels)
len(unique_signs)

In [8]:
labels = tf.keras.utils.to_categorical(labels, 43)
labels[0]

In [9]:
len(labels)

In [10]:
X = filenames
y = labels

X_train, X_val, y_train, y_val = train_test_split(X, y, test_size = 0.2, random_state = 42)
len(X_train), len(y_train), len(X_val), len(y_val)

In [11]:
IMG_SIZE = 32
def process_image(image_path):
    image = tf.io.read_file(image_path)
    image = tf.image.decode_png(image, channels=3)
    image = tf.image.convert_image_dtype(image, tf.float32)
    image = tf.image.resize(image, size=[IMG_SIZE, IMG_SIZE])
    return image

In [12]:
def get_image_label (image_path, label):
    image = process_image(image_path)
    return image, label

In [13]:
BATCH_SIZE = 64

def create_data_batches (X, y=None, batch_size=BATCH_SIZE, valid_data=False, test_data=False):
    if test_data:
        print("Creating test data batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X)))
        data_batch = data.map(process_image).batch(BATCH_SIZE)
    elif valid_data:
        print("Creating validation dataset batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    else:
        print("Creating training dataset batches...")
        data = tf.data.Dataset.from_tensor_slices((tf.constant(X), tf.constant(y)))
        data = data.shuffle(buffer_size=len(X))
        data_batch = data.map(get_image_label).batch(BATCH_SIZE)
    return data_batch

In [14]:
train_data = create_data_batches(X_train, y_train)
val_data = create_data_batches(X_val, y_val, valid_data=True)

In [15]:
train_data.element_spec, val_data.element_spec

In [16]:
def show_25_images (images, labels):
    plt.figure(figsize=(10,10))
    for i in range(25):
        ax = plt.subplot(5, 5, i+1)
        plt.imshow(images[i])
        plt.title(unique_signs[labels[i].argmax()])
        plt.axis("off")

In [17]:
train_images, train_labels = next(train_data.as_numpy_iterator())
show_25_images(train_images, train_labels)

In [18]:
val_images, val_labels = next(val_data.as_numpy_iterator())
show_25_images(val_images, val_labels)

In [19]:
INPUT_SHAPE = [IMG_SIZE, IMG_SIZE, 3]
OUTPUT_SHAPE = len(unique_signs)

In [20]:
def traffic_sign_net(input_shape):
    model = Sequential()
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu', input_shape=input_shape))
    model.add(Conv2D(filters=32, kernel_size=(5, 5), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(Conv2D(filters=64, kernel_size=(3, 3), activation='relu'))
    model.add(MaxPool2D(pool_size=(2, 2)))
    model.add(Dropout(rate=0.25))
    model.add(Flatten())
    model.add(Dense(256, activation='relu'))
    model.add(Dropout(rate=0.5))
    model.add(Dense(43, activation='softmax'))
    return model

In [21]:
def create_model(input_shape=INPUT_SHAPE, output_shape=OUTPUT_SHAPE):
    model = traffic_sign_net(input_shape=input_shape)
    print("Compiling the model")
    model.compile(
        loss = tf.keras.losses.CategoricalCrossentropy(),
        optimizer = tf.keras.optimizers.Adam(),
        metrics = ["accuracy"]
    )
    return model

In [22]:
model = create_model()
model.summary()

In [23]:
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=5)

In [38]:
NUM_EPOCHS = 10

In [39]:
def train_model():
    model = create_model()
    model.fit(x=train_data,
        epochs=NUM_EPOCHS,
        validation_data=val_data,
        validation_freq=1,
        callbacks=[early_stopping]
             )
    return model

In [40]:
model = train_model()

In [41]:
test_df = pd.read_csv('/kaggle/input/gtsrb-german-traffic-sign/Test.csv')
test_df = test_df.drop(['Width', 'Height', 'Roi.X1', 'Roi.Y1', 'Roi.X2', 'Roi.Y2'], axis=1)
test_df.head()

In [42]:
test_img_paths = ['/kaggle/input/gtsrb-german-traffic-sign/' + path for path in test_df['Path']]
test_img_paths[:10]

In [43]:
X_test = create_data_batches(test_img_paths, test_data=True)
y_test = list(test_df['ClassId'])
y_test[:10]

In [44]:
predictions = model.predict(X_test, verbose=1)

In [45]:
def get_pred_label(prediction_probabilities):
    return unique_signs[np.argmax(prediction_probabilities)]

In [46]:
pred_labels = []
for i in predictions:
    pred_labels.append(get_pred_label(i))
pred_labels[:10]

In [50]:
acc = accuracy_score(y_test, pred_labels)
acc